In [86]:
using DifferentialEquations
using PyCall
import Random
using PyPlot
using Statistics
using BenchmarkTools

In [173]:
function KM_model!(du, u, p, t)

    n = length(u)
    for i = 1:n
        du[i] = p[2][i] + p[1] * sum(p[3][:, i] .* sin.(u .- u[i]))
    end

    return du
end


function order(theta)
    
    n = length(theta)
    
    real_R = 0.0
    imag_R = 0.0
    
    for i = 1:n
        real_R += cos(theta[i])
        imag_R += sin(theta[i])
    end
    
    real_R /= n
    imag_R /= n
    r = sqrt(real_R * real_R + imag_R * imag_R)
    
    return r
end

order (generic function with 1 method)

In [174]:
Random.seed!(1234)

K = 0.7                      
N = 100                   
dt = 0.01                    
T = 200.0                   
T_trans = 0.0                
mu = 2.0                     
sigma = 0.1 
p = 1.0
R = zeros(length(K))

nx = pyimport("networkx")
np = pyimport("numpy")

G = nx.gnp_random_graph(N, p, seed=1)
adj = nx.to_numpy_array(G, dtype=np.int)

ind_transition = Int64(T_trans/dt)
theta_0 = rand(N) .* 2.0 .* pi .- pi
omega_0 = randn(N) .* sigma .+ mu 
tspan = (0.0, T)
p = [K[1]/N, omega_0, adj]

prob = ODEProblem(KM_model!, theta_0, tspan, p);

In [175]:
sol = solve(prob, saveat=0.01);
nstep = length(sol.t)
r = zeros(nstep)
for j =1:nstep
    r[j] = order(sol.u[j])
end
println(mean(r))

0.9445036095996592


In [176]:
@btime solve(prob, Tsit5(), saveat=0.01, abstol=1e-8,reltol=1e-8);

  126.602 ms (780582 allocations: 129.65 MiB)


In [ ]:
0.9445036095996592, 126.602 ms (780582 allocations: 129.65 MiB)


In [172]:
function KM_model!(du, u, p, t)

    n = length(u)
    for i = 1:n
        sumj = 0.0
        for j = 1:n
            sumj += p[3][j, i] * sin(u[j] - u[i])
        end
        du[i] = p[2][i] + p[1] * sumj
    end

    return du
end;